In [21]:
import numpy as np
import pandas as pd
import os
import sys
import zipfile
import subprocess

from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats
from tqdm.notebook import tqdm
from copy import deepcopy

import json

In [22]:
DATASET = 'ml-1m' 
RAW_PATH = os.path.join('./', DATASET)

RANDOM_SEED = 0
NEG_ITEMS = 99

# Load data

1. Load interaction data and item metadata
2. Filter out items with less than 5 interactions
3. Calculate basic statistics

In [19]:
# download data if not exists

if not os.path.exists(RAW_PATH):
    subprocess.call('mkdir ' + RAW_PATH, shell=True)
if not os.path.exists(os.path.join(RAW_PATH, DATASET + '.zip')):
    print('Downloading data into ' + RAW_PATH)
    #subprocess.call(
       # 'cd {} && curl -O http://files.grouplens.org/datasets/movielens/{}.zip'
        #.format(RAW_PATH, DATASET), shell=True)
    print('Unzip files...')
    f = zipfile.ZipFile(os.path.join(RAW_PATH, DATASET + '.zip'),'r') 
    for file in f.namelist():
        print("Extract %s"%(file))
        f.extract(file,RAW_PATH)
    f.close()

In [20]:
# read interaction data
interactions = []
user_freq, item_freq = dict(), dict()
file = os.path.join(RAW_PATH,"ratings.dat")
with open(file) as F:
    header = 0
    for line in tqdm(F):
        if header == 1:
            header = 0
            continue
        line = line.strip().split("::")
        uid, iid, rating, time = line[0], line[1], float(line[2]), float(line[3])
        if rating >= 4:
            label = 1
        else:
            label = 0
        interactions.append([uid,time,iid,label])
        if int(label)==1:
            user_freq[uid] = user_freq.get(uid,0)+1
            item_freq[iid] = item_freq.get(iid,0)+1

FileNotFoundError: [Errno 2] No such file or directory: './ml-1m\\ratings.dat'

In [23]:
# 5-core filtering
select_uid, select_iid = [],[]
while len(select_uid)<len(user_freq) or len(select_iid)<len(item_freq):
    select_uid, select_iid = [],[]
    for u in user_freq:
        if user_freq[u]>=5:
            select_uid.append(u)
    for i in item_freq:
        if item_freq[i]>=5:
            select_iid.append(i)
    print("User: %d/%d, Item: %d/%d"%(len(select_uid),len(user_freq),len(select_iid),len(item_freq)))

    select_uid = set(select_uid)
    select_iid = set(select_iid)
    user_freq, item_freq = dict(), dict()
    interactions_5core = []
    for line in tqdm(interactions):
        uid, iid, label = line[0], line[2], line[-1]
        if uid in select_uid and iid in select_iid:
            interactions_5core.append(line)
            if int(label)==1:
                user_freq[uid] = user_freq.get(uid,0)+1
                item_freq[iid] = item_freq.get(iid,0)+1
    interactions = interactions_5core

In [24]:
print("Selected Interactions: %d, Users: %d, Items: %d"%(len(interactions),len(select_uid),len(select_iid)))

Selected Interactions: 0, Users: 0, Items: 0


In [25]:
# Get timestamp
ts = []
for i in tqdm(range(len(interactions))):
    ts.append(datetime.fromtimestamp(interactions[i][1]))

0it [00:00, ?it/s]

In [27]:
# Construct and Save 5 core results with situation context
interaction_df = pd.DataFrame(interactions,columns = ["user_id","time","item_id","label"])
interaction_df['timestamp'] = ts
interaction_df['hour'] = interaction_df['timestamp'].apply(lambda x: x.hour)
interaction_df['weekday'] = interaction_df['timestamp'].apply(lambda x: x.weekday())
interaction_df['date'] = interaction_df['timestamp'].apply(lambda x: x.date())

def get_time_range(hour): # according to the Britannica dictionary
    # https://www.britannica.com/dictionary/eb/qa/parts-of-the-day-early-morning-late-morning-etc
    if hour>=5 and hour<=8:
        return 0
    if hour>8 and hour<11:
        return 1
    if hour>=11 and hour<=12:
        return 2
    if hour>12 and hour<=15:
        return 3
    if hour>15 and hour<=17:
        return 4
    if hour>=18 and hour<=19:
        return 5
    if hour>19 and hour<=21:
        return 6
    if hour>21:
        return 7
    return 8 # 0-4 am

interaction_df['period'] = interaction_df.hour.apply(lambda x: get_time_range(x))
min_date = interaction_df.date.min()
interaction_df['day'] = (interaction_df.date - min_date).apply(lambda x: x.days)

interaction_df.to_csv("interaction_5core.csv",index=False)
interaction_df["user_id"] = interaction_df["user_id"].astype(int)
interaction_df["item_id"] = interaction_df["item_id"].astype(int)


# Prepare data for CTR & Reranking task

1. Rename and organize all interaction features
2. Split dataset into training, validation, and test; Save interaction files
3. Assign impression ID (not necessary for CTR prediction)
4. Organize item metadata

In [28]:
CTR_PATH='./ML_1MCTR/'
os.makedirs(CTR_PATH,exist_ok=True)

In [30]:
# copy interaction file, rename and re-id all features
interaction_ctr = interaction_df.copy()
interaction_ctr.rename(columns={'hour':'c_hour_c','weekday':'c_weekday_c','period':'c_period_c','day':'c_day_f',
                              'user_id':'original_user_id','item_id': 'original_item_id'},
                     inplace=True)
user2newid_ctr = dict(zip(sorted(interaction_ctr.original_user_id.unique()), 
                      range(1,interaction_ctr.original_user_id.nunique()+1)))
interaction_ctr['user_id'] = interaction_ctr.original_user_id.apply(lambda x: user2newid_ctr[x])

item2newid_ctr = dict(zip(sorted(interaction_ctr.original_item_id.unique()), 
                      range(1,interaction_ctr.original_item_id.nunique()+1)))
interaction_ctr['item_id'] = interaction_ctr['original_item_id'].apply(lambda x: item2newid_ctr[x])
interaction_ctr.sort_values(by=['user_id','time'],inplace=True)
interaction_ctr = interaction_ctr.reset_index(drop=True)

nu2nid = dict()
ni2nid = dict()
for i in user2newid_ctr.keys():
    oi = int(i)
    nu2nid[oi] = user2newid_ctr[i]

for i in item2newid_ctr.keys():
    oi = int(i)
    ni2nid[oi] = item2newid_ctr[i]

json.dump(nu2nid,open(os.path.join(CTR_PATH,"user2newid.json"),'w'))
json.dump(ni2nid,open(os.path.join(CTR_PATH,"item2newid.json"),'w'))

In [31]:
# Count statistics
for col in interaction_ctr.columns:
    if col in ['user_id','item_id'] or col.startswith('c_'):
        print(col, interaction_ctr[col].nunique())

c_hour_c 0
c_weekday_c 0
c_period_c 0
c_day_f 0
user_id 0
item_id 0


In [32]:
# split training, validation, and test sets.
split_time1 = interaction_ctr.c_day_f.max() * 0.8
train = interaction_ctr.loc[interaction_ctr.c_day_f<=split_time1].copy()
val_test = interaction_ctr.loc[(interaction_ctr.c_day_f>split_time1)].copy()
split_time2 = interaction_ctr.c_day_f.max() * 0.9
val = val_test.loc[val_test.c_day_f<=split_time2].copy()
test = val_test.loc[val_test.c_day_f>split_time2].copy()

# Delete user&item in validation&test sets that not exist in training set
train_u, train_i = set(train.user_id.unique()), set(train.item_id.unique())
val_sel = val.loc[(val.user_id.isin(train_u))&(val.item_id.isin(train_i))].copy()
test_sel = test.loc[(test.user_id.isin(train_u))&(test.item_id.isin(train_i))].copy()
print("Train user: %d, item: %d"%(len(train_u),len(train_i)))
print("Validation user: %d, item:%d"%(val_sel.user_id.nunique(),val_sel.item_id.nunique()))
print("Test user: %d, item:%d"%(test_sel.user_id.nunique(),test_sel.item_id.nunique()))
train.label.sum(),train.label.mean(),val_sel.label.sum(),val_sel.label.mean(),test_sel.label.sum(),test_sel.label.mean()

Train user: 0, item: 0
Validation user: 0, item:0
Test user: 0, item:0


(0, nan, 0, nan, 0, nan)

In [33]:
test_sel.shape

(0, 12)

In [34]:
# Assign impression ids
for interaction_partial in [train,val_sel,test_sel]:
    interaction_partial['last_user_id'] = interaction_partial['user_id'].shift(1)
    impression_ids = []
    impression_len = 0
    current_impid = 0
    max_imp_len = 20
    for uid, last_uid in tqdm(interaction_partial[['user_id','last_user_id']].to_numpy()):
        if uid == last_uid:
            if impression_len >= max_imp_len:
                current_impid += 1
                impression_len = 1
            else:
                impression_len += 1
            impression_ids.append(current_impid)
        else:
            current_impid += 1
            impression_len = 1
            impression_ids.append(current_impid)
    interaction_partial.loc[:,'impression_id'] = impression_ids

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [35]:
# Save interaction data
select_columns = ['user_id','item_id','time','label','c_hour_c','c_weekday_c','c_period_c','c_day_f','impression_id']
train[select_columns].to_csv(os.path.join(CTR_PATH,'train.csv'),sep="\t",index=False)
val_sel[select_columns].to_csv(os.path.join(CTR_PATH,'dev.csv'),sep="\t",index=False)
test_sel[select_columns].to_csv(os.path.join(CTR_PATH,'test.csv'),sep="\t",index=False)

In [37]:
# organize & save item metadata
item_meta = pd.read_csv(os.path.join(DATASET, "movies.dat"),
            sep='::',names=['movieId','title','genres'],encoding='latin-1',engine='python') # columns: movieId,title,genres
item_select = item_meta.loc[item_meta.movieId.astype(str).isin(interaction_ctr.original_item_id.unique())].copy()
item_select['item_id'] = item_select.movieId.apply(lambda x: item2newid_ctr[str(x)])
genres2id = dict(zip(sorted(item_select.genres.unique()),range(1,item_select.genres.nunique()+1)))
item_select['i_genre_c'] = item_select['genres'].apply(lambda x: genres2id[x])
title2id = dict(zip(sorted(item_select.title.unique()),range(1,item_select.title.nunique()+1)))
item_select['i_title_c'] = item_select['title'].apply(lambda x: title2id[x])

item_select[['item_id','i_genre_c','i_title_c']].to_csv(
    os.path.join(CTR_PATH,'item_meta.csv'),sep="\t",index=False)

# Prepare data for Top-k Recommendation Task
1. Rename all interaction features
2. Split dataset into training, validation, and test
3. Re-assign IDs to user, item, and context; Save interaction files
4. Organize item metadata

In [38]:
TOPK_PATH='./ML_1MTOPK/'
os.makedirs(TOPK_PATH,exist_ok=True)

In [39]:
# copy & rename columns
interaction_pos = interaction_df.loc[interaction_df.label==1].copy() # retain positive interactions
interaction_pos.rename(columns={'hour':'c_hour_c','weekday':'c_weekday_c','period':'c_period_c','day':'c_day_f',
                              'user_id':'original_user_id'}, inplace=True)

In [42]:
train_u, train_i = set(train.original_user_id.unique()), set(train.news_id.unique())
val_sel = val.loc[(val.original_user_id.isin(train_u))&(val.news_id.isin(train_i))].copy()
test_sel = test.loc[(test.original_user_id.isin(train_u))&(test.news_id.isin(train_i))].copy()
print("Train user: %d, item: %d"%(len(train_u),len(train_i)))
print("Validation user: %d, item:%d"%(val_sel.original_user_id.nunique(),val_sel.news_id.nunique()))
print("Test user: %d, item:%d"%(test_sel.original_user_id.nunique(),test_sel.news_id.nunique()))
train.label.sum(),train.label.mean(),val_sel.label.sum(),val_sel.label.mean(),test_sel.label.sum(),test_sel.label.mean()

AttributeError: 'DataFrame' object has no attribute 'news_id'

In [ ]:
# Assign ids for users and items (to generate continous ids)
all_df = pd.concat([train,val_sel,test_sel],axis=0)
user2newid_topk = dict(zip(sorted(all_df.original_user_id.unique()), 
                      range(1,all_df.original_user_id.nunique()+1)))
 
for df in [train,val_sel,test_sel]:
    df['user_id'] = df.original_user_id.apply(lambda x: user2newid_topk[x])

item2newid_topk = dict(zip(sorted(all_df.news_id.unique()), 
                      range(1,all_df.news_id.nunique()+1)))
for df in [train,val_sel,test_sel]:
    df['item_id'] = df['news_id'].apply(lambda x: item2newid_topk[x])

all_df['user_id'] = all_df.original_user_id.apply(lambda x: user2newid_topk[x])
all_df['item_id'] = all_df['news_id'].apply(lambda x: item2newid_topk[x])

In [ ]:
nu2nid = dict()
ni2nid = dict()
for i in user2newid_topk.keys():
    oi = int(i)
    nu2nid[oi] = user2newid_topk[i]

for i in item2newid_topk.keys():
    oi = int(i)
    ni2nid[oi] = item2newid_topk[i]
json.dump(nu2nid,open(os.path.join(TOPK_PATH,"user2newid.json"),'w'))
json.dump(ni2nid,open(os.path.join(TOPK_PATH,"item2newid.json"),'w'))

In [ ]:
# generate negative items
def generate_negative(data_df,all_items,clicked_item_set,random_seed,neg_item_num=99):
    np.random.seed(random_seed)
    neg_items = np.random.choice(all_items, (len(data_df),neg_item_num))
    for i, uid in tqdm(enumerate(data_df['user_id'].values)):
        user_clicked = clicked_item_set[uid]
        for j in range(len(neg_items[i])):
            while neg_items[i][j] in user_clicked|set(neg_items[i][:j]):
                neg_items[i][j] = np.random.choice(all_items, 1)
    return neg_items.tolist()

clicked_item_set = dict()
for user_id, seq_df in all_df.groupby('user_id'):
    clicked_item_set[user_id] = set(seq_df['item_id'].values.tolist())
all_items = all_df.item_id.unique()
val_sel['neg_items'] = generate_negative(val_sel,all_items,clicked_item_set,random_seed=1)
test_sel['neg_items'] = generate_negative(test_sel,all_items,clicked_item_set,random_seed=2)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
select_columns = ['user_id','item_id','time','c_hour_c','c_weekday_c','c_period_c','c_day_f']
train[select_columns].to_csv(os.path.join(TOPK_PATH,'train.csv'),sep="\t",index=False)
val_sel[select_columns+['neg_items']].to_csv(os.path.join(TOPK_PATH,'dev.csv'),sep="\t",index=False)
test_sel[select_columns+['neg_items']].to_csv(os.path.join(TOPK_PATH,'test.csv'),sep="\t",index=False)

In [ ]:
# organize & save item metadata
item_meta = pd.read_csv(os.path.join(DATASET, "movies.dat"),
            sep='::',names=['movieId','title','genres'],encoding='latin-1',engine='python') # columns: movieId,title,genres
item_select = item_meta.loc[item_meta.movieId.isin(interaction_pos.news_id.unique())].copy()
item_select['item_id'] = item_select.movieId.apply(lambda x: item2newid_topk[x])
genres2id = dict(zip(sorted(item_select.genres.unique()),range(1,item_select.genres.nunique()+1)))
item_select['i_genre_c'] = item_select['genres'].apply(lambda x: genres2id[x])
title2id = dict(zip(sorted(item_select.title.unique()),range(1,item_select.title.nunique()+1)))
item_select['i_title_c'] = item_select['title'].apply(lambda x: title2id[x])

item_select[['item_id','i_genre_c','i_title_c']].to_csv(
    os.path.join(TOPK_PATH,'item_meta.csv'),sep="\t",index=False)

In [44]:
import pandas as pd
import os

# 1. 直接使用你提供的绝对路径 (最稳妥)
# 原始数据位置 (存放 ratings.dat 的地方)
RAW_PATH = r"D:\jianq\robot_learn\project\ReChorus\data\MovieLens_1M\ml-1m"

# 目标保存位置 (存放 train.csv 的地方，也就是上一级目录)
CTR_PATH = r"D:\jianq\robot_learn\project\ReChorus\data\MovieLens_1M"

# 拼接文件名
DATA_FILE = os.path.join(RAW_PATH, "ratings.dat")

print(f"Reading data from: {DATA_FILE}")

# 2. 读取数据
try:
    df = pd.read_csv(DATA_FILE, sep='::', header=None, engine='python', 
                     names=['user_id', 'item_id', 'rating', 'timestamp'])
    print("✅ 数据读取成功！")
except FileNotFoundError:
    print(f"❌ 依然报错？请确认这个路径下是否有 ratings.dat: {DATA_FILE}")
    raise

# 3. 数据清洗 (只保留评分 >= 4)
df = df[df['rating'] >= 4].copy()
print(f"有效数据量: {len(df)}")

# 4. ID 映射
user2id = {uid: i+1 for i, uid in enumerate(sorted(df['user_id'].unique()))}
item2id = {iid: i+1 for i, iid in enumerate(sorted(df['item_id'].unique()))}
df['user_id'] = df['user_id'].map(user2id)
df['item_id'] = df['item_id'].map(item2id)

# 5. 时间切分
df['date'] = pd.to_datetime(df['timestamp'], unit='s')
min_date = df['date'].min()
df['day'] = (df['date'] - min_date).dt.days

split_day_1 = int(df['day'].max() * 0.8)
split_day_2 = int(df['day'].max() * 0.9)

train = df[df['day'] <= split_day_1].copy()
val_test = df[df['day'] > split_day_1].copy()
val = val_test[val_test['day'] <= split_day_2].copy()
test = val_test[val_test['day'] > split_day_2].copy()

# 6. 冷启动过滤
train_users = set(train['user_id'].unique())
train_items = set(train['item_id'].unique())
val = val[val['user_id'].isin(train_users) & val['item_id'].isin(train_items)]
test = test[test['user_id'].isin(train_users) & test['item_id'].isin(train_items)]

# 7. 保存 CSV
train.rename(columns={'timestamp': 'time'}, inplace=True)
val.rename(columns={'timestamp': 'time'}, inplace=True)
test.rename(columns={'timestamp': 'time'}, inplace=True)

train[['user_id', 'item_id', 'time']].to_csv(os.path.join(CTR_PATH, 'train.csv'), index=False)
val[['user_id', 'item_id', 'time']].to_csv(os.path.join(CTR_PATH, 'validation.csv'), index=False)
test[['user_id', 'item_id', 'time']].to_csv(os.path.join(CTR_PATH, 'test.csv'), index=False)

print("-" * 30)
print("✅ SUCCESS! 成功生成 train.csv, validation.csv, test.csv")
print(f"文件位置: {CTR_PATH}")

Reading data from: D:\jianq\robot_learn\project\ReChorus\data\MovieLens_1M\ml-1m\ratings.dat
✅ 数据读取成功！
有效数据量: 575281
------------------------------
✅ SUCCESS! 成功生成 train.csv, validation.csv, test.csv
文件位置: D:\jianq\robot_learn\project\ReChorus\data\MovieLens_1M


In [1]:
import pandas as pd
import os

# 设置数据路径
CTR_PATH = r"D:\jianq\robot_learn\project\ReChorus\data\MovieLens_1M"

# 需要修复的三个文件
files = ['train.csv', 'dev.csv', 'test.csv']

print(f"正在将文件格式从 [逗号分隔] 转换为 [Tab分隔] ...")
print(f"目标路径: {CTR_PATH}")

for file_name in files:
    file_path = os.path.join(CTR_PATH, file_name)
    
    if os.path.exists(file_path):
        # 1. 用逗号读取 (Current Format)
        df = pd.read_csv(file_path, sep=',')
        
        # 2. 用 Tab 保存 (ReChorus Format)
        df.to_csv(file_path, sep='\t', index=False)
        
        print(f"✅ 已修复: {file_name}")
    else:
        print(f"❌ 找不到文件: {file_name}")
        if file_name == 'dev.csv':
            print("   (提示: 请检查你是否忘记把 validation.csv 重命名为 dev.csv ?)")

print("-" * 30)
print("格式修复完成！现在可以去跑实验了！")

正在将文件格式从 [逗号分隔] 转换为 [Tab分隔] ...
目标路径: D:\jianq\robot_learn\project\ReChorus\data\MovieLens_1M
✅ 已修复: train.csv
✅ 已修复: dev.csv
✅ 已修复: test.csv
------------------------------
格式修复完成！现在可以去跑实验了！
